In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [2]:
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time
import timeit #imports timeit module
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
start_time = timeit.default_timer() #defines start time so computational time can be calculated
print("Hello World")
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

Hello World
---Run time is 0.00011959999999788806 seconds ---



In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import statistics
import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
import wfdb
import numpy as np
import pandas as pd

In [4]:
# import os
# cwd = os.getcwd()
# print(cwd)
# os.chdir('/Users/chenc/000---STT450-Jupyter Notebook/000--STT450-550/') 
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:21<00:00,  1.17it/s]


In [5]:
# df3 = pd.read_csv('Data/Carseats.csv')
# #df3.info()
# print(); df3.head(5)
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
features_maindf = pd.concat(features_df_list, ignore_index = True)

In [6]:
# df3['High'] = df3.Sales.map(lambda x: 1 if x>8 else 0)
# df3.ShelveLoc = pd.factorize(df3.ShelveLoc)[0]
# df3.Urban = df3.Urban.map({'No':0, 'Yes':1})
# df3.US = df3.US.map({'No':0, 'Yes':1})
# #df3.info()
# print(); df3.head(5)
#X = df3.drop(['Sales', 'High'], axis = 1)
#y = df3.High
##print(X.shape)
#print(y.shape)

In [7]:
x_values = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn','Trans_AppEn']
y_values = ['Rhythm']
#features_df_list[0]

### Decision Tree

criterion{“gini”, “entropy”}, default=”gini”

In [8]:
# #for df in features_df_list:
# #model = DecisionTreeClassifier(max_depth = 6) #criterion='entropy'
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# max_depth_list = []
# n_estimators_list = []
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     temp_model = DecisionTreeClassifier(max_depth = 6)    
#     param_distributions = {'max_depth': randint(7,9),
#                           'nestim'}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])
    
#     model = DecisionTreeClassifier(max_depth = 8) #criterion='entropy'
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 
# elapsed = timeit.default_timer() - start_time #gives total computation time
# print("---Run time is %s seconds ---" % elapsed) #prints computation time
# print()

# print('Accuracy of each fold: \n {}'.format(acc_score))
# print()
# print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
# print()
# print('Std of accuracy : \n{}'.format(np.std(acc_score)))

In [9]:
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(acc_score)
# print(elapsed)
# #print(n_estimators_list)
# # print(max_depth_list)

# # temp_list = []
# # temp_idx = []
# # for acc in acc_score:
# #     if acc >= 0.95:
# #         temp_list.append(acc)
# #         temp_idx.append(acc_score.index(acc))
# # for idx in temp_idx:
# #     print(max_depth_list[idx])
    
# # statistics.mode(max_depth_list)

In [10]:
# [[22543  2200]
#  [ 2067 18158]]
#               precision    recall  f1-score   support

#            0       0.92      0.91      0.91     24743
#            1       0.89      0.90      0.89     20225

#     accuracy                           0.91     44968
#    macro avg       0.90      0.90      0.90     44968
# weighted avg       0.91      0.91      0.91     44968

In [11]:
# [[22796  1947]
#  [ 1763 18462]]
#               precision    recall  f1-score   support

#            0       0.93      0.92      0.92     24743
#            1       0.90      0.91      0.91     20225

#     accuracy                           0.92     44968
#    macro avg       0.92      0.92      0.92     44968
# weighted avg       0.92      0.92      0.92     44968

In [12]:
# Avg accuracy : 
# 0.9225946312949485

# Std of accuracy : 
# 0.10175840774589326

# [[22670  2073]
#  [ 1762 18463]]
#               precision    recall  f1-score   support

#            0       0.93      0.92      0.92     24743
#            1       0.90      0.91      0.91     20225

#     accuracy                           0.91     44968
#    macro avg       0.91      0.91      0.91     44968
# weighted avg       0.91      0.91      0.91     44968

# Bagging: use p for classification

In [13]:
statistics.mode([13, 11, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 13, 12, 13, 13, 11, 11, 13, 11, 13, 11])
statistics.mode([127, 130, 127, 132, 132, 132, 127, 127, 132, 122, 127, 132, 128, 116, 134, 127, 127, 130, 138, 127, 138, 122, 130])

127

In [15]:
# for acc in acc_score:
#     if acc >= 0.95:
#         temp_list.append(acc)
#         temp_idx.append(acc_score.index(acc))
# for idx in temp_idx:
#     n_estimators_best.append(n_estimators_list[idx])
#     max_depth_best.append(max_depth_list[idx])

In [ ]:

#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
# n_estimators_list = []
# max_depth_list = []
import statistics
#mode_depth = statistics.mode(max_depth_best)
#mode_estimators = statistics.mode(n_estimators_best)
#confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]
    
#     temp_model = RandomForestClassifier(max_features=16,random_state = 2)    
#     param_distributions = {'max_depth': randint(11,14),
#                           'n_estimators': randint(115,139)}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])
    
    model = RandomForestClassifier(max_features = 16, random_state = 2, max_depth=13, 
                                   n_estimators=127)
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(acc_score)
print(elapsed)
print(n_estimators_list)
print(max_depth_list)
# temp_list = []
# temp_idx = []
# for acc in acc_score:
#     if acc >= 0.95:
#         temp_list.append(acc)
#         temp_idx.append(acc_score.index(acc))
# for idx in temp_idx:
#     print(n_estimators_list[idx])
#     print(max_depth_list[idx])
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(acc_score)
# print(elapsed)
# print(n_estimators_list)
# print(max_depth_list)

# temp_list = []
# temp_idx = []
# for acc in acc_score:
#     if acc >= 0.95:
#         temp_list.append(acc)
#         temp_idx.append(acc_score.index(acc))
# for idx in temp_idx:
#     print(n_estimators_list[idx])
#     print(max_depth_list[idx])

In [ ]:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
# n_estimators_list = []
# max_depth_list = []
import statistics
#mode_depth = statistics.mode(max_depth_best)
#mode_estimators = statistics.mode(n_estimators_best)
#confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]
    
#     temp_model = RandomForestClassifier(max_features=16,random_state = 2)    
#     param_distributions = {'max_depth': randint(11,14),
#                           'n_estimators': randint(115,139)}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])
    
    model = RandomForestClassifier(max_features = 16, random_state = 2, max_depth=5)
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(acc_score)
print(elapsed)

In [ ]:
import statistics
statistics.mode(n_estimators_list)
statistics.mode(max_depth_list)

In [ ]:
# [[23029  1714]
#  [ 1649 18576]]
#               precision    recall  f1-score   support

#            0       0.93      0.93      0.93     24743
#            1       0.92      0.92      0.92     20225

#     accuracy                           0.93     44968
#    macro avg       0.92      0.92      0.92     44968
# weighted avg       0.93      0.93      0.93     44968
# ---Run time is 27743.89577069995 seconds ---

# Accuracy of each fold: 
#  [0.914187643020595, 0.9509127789046653, 0.9955919395465995, 0.9250599520383693, 0.9973876698014629, 0.9565922920892495, 0.79542269967305, 0.9979577944179714, 0.9422690763052208, 0.9680264608599779, 0.9813974591651543, 0.9885139985642498, 0.8335600907029479, 0.9496815286624204, 0.6186511240632806, 0.9858218874612317, 0.9876627827278959, 0.9855741488747836, 0.7324598478444633, 0.971412864211105, 0.9940119760479041, 0.997489014438167, 0.9936575052854123]

# Avg accuracy : 
# 0.9331870667263553

# Std of accuracy : 
# 0.09555730626481958

In [ ]:
#help(confusion_matrix)

In [ ]:
# crossvalidation = KFold(n_splits=23)
# #for df in features_df_list:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     model = RandomForestClassifier(max_features = 10, random_state = 2)
#     model.fit(X_train,y_train.values.ravel())

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 


# model = RandomForestClassifier(max_features = 16, random_state = 2)
# scores = cross_val_score(model, X, y, cv=crossvalidation)
# print('Accuracy of each fold: \n {}'.format(scores))
# print()
# print("Avg accuracy: {}".format(scores.mean()))

### An AdaBoost classifier.

The argument `n_estimators = 500` indicates that we want 500 trees

class sklearn.ensemble.AdaBoostClassifier(base_estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None)

In [ ]:
# crossvalidation = KFold(n_splits=23)
# #for df in features_df_list:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 

In [ ]:
# print(acc_score)
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(confusion_matrix_df)

### Gradient Boosting for classification

In [ ]:
# crossvalidation = KFold(n_splits=23)
# #for df in features_df_list:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# #confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     model =  GradientBoostingClassifier(n_estimators = 500, 
#                                            learning_rate = 0.1, 
#                                            max_depth = 4, 
#                                            random_state = 2)
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
#     #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 

In [ ]:
# print(acc_score)
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(confusion_matrix_df)

In [8]:
# df
x_values = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn']
# for temp in x_values:
        
for temp in x_values:    
    
    x_value = [temp]
    print(x_value)
    print(type(x_value))

['StoS']
<class 'list'>
['StoR']
<class 'list'>
['StoL']
<class 'list'>
['RtoS']
<class 'list'>
['RtoR']
<class 'list'>
['RtoL']
<class 'list'>
['LtoS']
<class 'list'>
['LtoR']
<class 'list'>
['LtoL']
<class 'list'>
['STD']
<class 'list'>
['CoefVar']
<class 'list'>
['Range']
<class 'list'>
['Volatility']
<class 'list'>
['ShEn']
<class 'list'>
['AppEn']
<class 'list'>
['SampEn']
<class 'list'>


In [9]:
def confusion_calcs(confusion_matrix):
    temp_arr = confusion_matrix
    tn = temp_arr[0]
    fp = temp_arr[1]
    fn = temp_arr[2]
    tp = temp_arr[3]
    sensitivity = tp / (tp + fn)
    recall = sensitivity
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    accuracy = (tp+tn) / (tp+tn+fn+fp)
    f1 = 2*precision*recall/(precision+recall)
    return [accuracy, sensitivity, specificity, precision, f1]

In [10]:
df = features_maindf
import statistics
    

all_features = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn','Trans_AppEn']
dataScores = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

        
for score_idx, temp in tqdm(enumerate(all_features)):    
    
    x_values = [temp]
    print(x_values)
    
    
    acc_score = [];
    Truth = [];
    Output = [];


    for idx, subject in tqdm(enumerate(good_list)):

        train_index = ~df['Subject'].str.contains(subject)
        test_index = df['Subject'].str.contains(subject)
        X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
        y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

    #     temp_model = RandomForestClassifier(max_features=16,random_state = 2)    
    #     param_distributions = {'max_depth': randint(11,14),
    #                           'n_estimators': randint(115,139)}

    #     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
    #     search.fit(X_train, y_train)

    #     n_estimators_list.append(search.best_params_['n_estimators'])
    #     max_depth_list.append(search.best_params_['max_depth'])

        model = RandomForestClassifier(max_features = 1, random_state = 2, max_depth=5, n_estimators = 127)
        model.fit(X_train,y_train)

        pred_values = model.predict(X_test)
        pred_prob = model.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)
        #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
        Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values); ## it is a list 
#     elapsed = timeit.default_timer() - start_time #gives total computation time
#     print("---Run time is %s seconds ---" % elapsed) #prints computation time
#     print()

    print('Accuracy of each fold: \n {}'.format(acc_score))
    print()
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
    print()
    print('Std of accuracy : \n{}'.format(np.std(acc_score)))
    print(confusion_matrix(Truth, Output))
    print(classification_report(Truth, Output))
    print(acc_score)
    print(elapsed)
    
    temp_array = confusion_matrix(Truth, Output)
    joint = temp_array[0].tolist() + temp_array[1].tolist()
    temp_scores = confusion_calcs(joint)
    
    #dataScores[score_idx].append(all_features[score_idx])
    #dataScores[score_idx].append(acc_score)
    dataScores[score_idx].append(np.mean(acc_score))
    dataScores[score_idx].append(np.std(acc_score))
    
    for confusionScore in temp_scores:
        dataScores[score_idx].append(confusionScore)

0it [00:00, ?it/s]
0it [00:00, ?it/s]

['StoS']



1it [00:00,  1.57it/s]
2it [00:01,  1.56it/s]
3it [00:01,  1.57it/s]
4it [00:02,  1.58it/s]
5it [00:03,  1.58it/s]
6it [00:03,  1.59it/s]
7it [00:04,  1.54it/s]
8it [00:05,  1.44it/s]
9it [00:05,  1.46it/s]
10it [00:06,  1.50it/s]
11it [00:07,  1.53it/s]
12it [00:07,  1.52it/s]
13it [00:08,  1.54it/s]
14it [00:09,  1.54it/s]
15it [00:09,  1.43it/s]
16it [00:10,  1.40it/s]
17it [00:11,  1.43it/s]
18it [00:12,  1.38it/s]
19it [00:12,  1.37it/s]
20it [00:13,  1.35it/s]
21it [00:14,  1.33it/s]
22it [00:15,  1.38it/s]
23it [00:15,  1.45it/s]
1it [00:15, 15.98s/it]
0it [00:00, ?it/s]

Accuracy of each fold: 
 [0.8483981693363845, 0.24178498985801217, 0.9861460957178841, 0.8920863309352518, 0.8359456635318704, 0.09776876267748479, 0.6202709014479215, 0.9904697072838666, 0.7399598393574297, 0.7276736493936052, 0.6311252268602541, 0.9849246231155779, 0.6063492063492063, 0.3159235668789809, 0.5645295587010825, 0.8817013735046522, 0.9129540781357094, 0.31217541834968265, 0.2569737954353339, 0.8290269378779549, 0.8387510692899914, 0.9648462021343377, 0.852431289640592]

Avg accuracy : 
0.6927050632962202

Std of accuracy : 
0.2669211118961723
[[15295  9448]
 [ 5524 14701]]
              precision    recall  f1-score   support

           0       0.73      0.62      0.67     24743
           1       0.61      0.73      0.66     20225

    accuracy                           0.67     44968
   macro avg       0.67      0.67      0.67     44968
weighted avg       0.68      0.67      0.67     44968

[0.8483981693363845, 0.24178498985801217, 0.9861460957178841, 0.892086330935251


1it [00:00,  1.66it/s]
2it [00:01,  1.62it/s]
3it [00:01,  1.64it/s]
4it [00:02,  1.64it/s]
5it [00:03,  1.64it/s]
6it [00:03,  1.65it/s]
7it [00:04,  1.58it/s]
8it [00:04,  1.59it/s]
9it [00:05,  1.58it/s]
10it [00:06,  1.57it/s]
11it [00:06,  1.56it/s]
12it [00:07,  1.57it/s]
13it [00:08,  1.57it/s]
14it [00:08,  1.59it/s]
15it [00:09,  1.59it/s]
16it [00:10,  1.60it/s]
17it [00:10,  1.61it/s]
18it [00:11,  1.55it/s]
19it [00:11,  1.56it/s]
20it [00:12,  1.56it/s]
21it [00:13,  1.58it/s]
22it [00:13,  1.53it/s]
23it [00:14,  1.58it/s]
2it [00:30, 15.19s/it]
0it [00:00, ?it/s]

...

Accuracy of each fold: 
 [0.8827231121281465, 0.9387423935091278, 0.993073047858942, 0.6091127098321343, 0.9801462904911181, 0.9225152129817444, 0.8958430639887903, 0.9870660313138189, 0.9252008032128514, 0.8699007717750827, 0.9804900181488203, 0.9633883704235463, 0.7183673469387755, 1.0, 0.2951706910907577, 0.8413823659725299, 0.9753255654557916, 0.9976918638199653, 0.6001690617075233, 0.9466739967014843, 0.9687767322497861, 0.933458882611425, 0.9446088794926004]

Avg accuracy : 
0.876949009204555

Std of accuracy : 
0.16639420138766964
[[21380  3363]
 [ 2671 17554]]
              precision    recall  f1-score   support

           0       0.89      0.86      0.88     24743
           1       0.84      0.87      0.85     20225

    accuracy                           0.87     44968
   macro avg       0.86      0.87      0.86     44968
weighted avg       0.87      0.87      0.87     44968

[0.8827231121281465, 0.9387423935091278, 0.993073047858942, 0.6091127098321343, 0.9801462904911181


1it [00:01,  1.67s/it]
2it [00:03,  1.66s/it]
3it [00:05,  1.67s/it]
4it [00:06,  1.71s/it]
5it [00:08,  1.68s/it]
6it [00:10,  1.67s/it]
7it [00:11,  1.65s/it]
8it [00:13,  1.67s/it]
9it [00:14,  1.66s/it]
10it [00:16,  1.66s/it]
11it [00:18,  1.66s/it]
12it [00:20,  1.68s/it]
13it [00:21,  1.67s/it]
14it [00:23,  1.66s/it]
15it [00:24,  1.64s/it]
16it [00:26,  1.65s/it]
17it [00:28,  1.66s/it]
18it [00:29,  1.64s/it]
19it [00:31,  1.64s/it]
20it [00:33,  1.65s/it]
21it [00:34,  1.64s/it]
22it [00:36,  1.64s/it]
23it [00:38,  1.65s/it]
16it [07:12, 37.59s/it]
0it [00:00, ?it/s]

Accuracy of each fold: 
 [0.8832951945080092, 0.947261663286004, 0.9924433249370277, 0.6097122302158273, 0.9811912225705329, 0.9233265720081136, 0.9065857076132648, 0.9877467665078284, 0.927710843373494, 0.7932745314222712, 0.9818511796733213, 0.9540559942569993, 0.6975056689342404, 1.0, 0.3659450457951707, 0.9153743907842269, 0.9664153529814942, 0.9976918638199653, 0.5743871513102282, 0.9516217702034084, 0.9816082121471343, 0.9133709981167608, 0.9720930232558139]

Avg accuracy : 
0.8793247264226581

Std of accuracy : 
0.16060848533605346
[[21006  3737]
 [ 2102 18123]]
              precision    recall  f1-score   support

           0       0.91      0.85      0.88     24743
           1       0.83      0.90      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968

[0.8832951945080092, 0.947261663286004, 0.9924433249370277, 0.6097122302158273, 0.981191222570532


1it [00:00,  1.11it/s]
2it [00:01,  1.13it/s]
3it [00:02,  1.14it/s]
4it [00:03,  1.15it/s]
5it [00:04,  1.15it/s]
6it [00:05,  1.15it/s]
7it [00:06,  1.12it/s]
8it [00:07,  1.13it/s]
9it [00:07,  1.14it/s]
10it [00:08,  1.15it/s]
11it [00:09,  1.15it/s]
12it [00:10,  1.15it/s]
13it [00:11,  1.14it/s]
14it [00:12,  1.15it/s]
15it [00:13,  1.14it/s]
16it [00:14,  1.12it/s]
17it [00:14,  1.12it/s]
18it [00:15,  1.13it/s]
19it [00:16,  1.13it/s]
20it [00:17,  1.14it/s]
21it [00:18,  1.14it/s]
22it [00:19,  1.15it/s]
23it [00:20,  1.12it/s]
17it [07:32, 26.64s/it]

Accuracy of each fold: 
 [0.898741418764302, 0.9237322515212981, 0.9899244332493703, 0.8267386091127098, 0.9258098223615465, 0.9022312373225152, 0.8388603456328818, 0.9863852961198094, 0.8724899598393574, 0.814773980154355, 0.8933756805807622, 0.9562096195262024, 0.728344671201814, 0.8866242038216561, 0.6419650291423813, 0.9264510412051395, 0.9458533241946538, 0.8886324293133295, 0.5714285714285714, 0.9395272127542605, 0.9174508126603935, 0.9353421217827997, 0.9361522198731501]

Avg accuracy : 
0.8759584474592722

Std of accuracy : 
0.10115848245999051
[[21486  3257]
 [ 2693 17532]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88     24743
           1       0.84      0.87      0.85     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968

[0.898741418764302, 0.9237322515212981, 0.9899244332493703, 0.8267386091127098, 0.9

In [11]:
# dataScores[score_idx].append(acc_score)
#     dataScores[score_idx].append(np.mean(acc_score))
#     dataScores[score_idx].append(np.std(acc_score))
    
#     for confusionScore in temp_scores:
#         dataScores[score_idx].append(confusionScore)
# return [accuracy, sensitivity, specificity, precision, f1]

In [12]:
# len(dataScores[0][1:8])
# dataScores

In [13]:

    
export_df = pd.DataFrame(dataScores,columns=['Mean Acc.', 'SD of Acc.', 'Total Acc.', 'Sensitivity', 
                                             'Specificity','Precision','f1'])

export_df = export_df.set_index(pd.Index(['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn','Trans_AppEn']))

In [14]:
export_df

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845303,0.132344,0.836217,0.841928,0.831548,0.803359,0.822192
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822794,0.118022,0.820406,0.856910,0.790567,0.769822,0.811035
LtoL,0.692080,0.243431,0.670388,0.306354,0.967951,0.886536,0.455354
STD,0.787988,0.195803,0.786226,0.874363,0.714182,0.714332,0.786288


In [15]:
export_df_highlight = export_df.copy()

export_df_highlight.style.highlight_max(color = 'lightgreen', axis = 0)


,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845303,0.132344,0.836217,0.841928,0.831548,0.803359,0.822192
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822794,0.118022,0.820406,0.856910,0.790567,0.769822,0.811035
LtoL,0.692080,0.243431,0.670388,0.306354,0.967951,0.886536,0.455354
STD,0.787988,0.195803,0.786226,0.874363,0.714182,0.714332,0.786288


In [16]:
export_df_highlight.style.highlight_min(color = 'lightcoral', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845303,0.132344,0.836217,0.841928,0.831548,0.803359,0.822192
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822794,0.118022,0.820406,0.856910,0.790567,0.769822,0.811035
LtoL,0.692080,0.243431,0.670388,0.306354,0.967951,0.886536,0.455354
STD,0.787988,0.195803,0.786226,0.874363,0.714182,0.714332,0.786288


In [17]:
export_df.to_csv(f'individual-features/Bagging.csv')